In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import re

MODEL_PATH = "./results/dapt_tapt_gnri/tapt"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForMaskedLM.from_pretrained(MODEL_PATH).to("cuda" if torch.cuda.is_available() else "cpu")

text = """
唯王元年，六月既朢乙亥，王在周穆王大[mask]。[mask]若曰：曶，命女更乃且考𤔲卜事，賜汝赤緼[mask]、□，用事，王在𫑕㡴，井叔賜曶赤金、鬱，曶受休命于王，曶用茲金作朕文考宄伯䵼牛鼎，曶其[mask][mask]用祀，子子孫孫其永寶。唯王四月既生霸，辰在丁酉，井叔在異，為[mask] [mask]，使厥小子究，以限訟于井叔，我既贖汝五[mask] [mask]。父用匹馬、束絲，限許曰：𬇊則俾我償馬。效[mask] [mask]俾復厥絲束，𫎛、效父廼許𬥱，曰：于王參門，[mask] [mask] 木梬，用徵誕贖茲五夫，用百鋝，非出五夫，[mask] [mask] 䛌，廼𬇎有䛌眔𫦠金，井叔曰：裁，王人廼贖[mask] [mask]，不逆付，曶毋俾弍，曶則拜𩒨首，受茲五[mask]，曰𬯂、曰恒、曰劦、曰𪝶、曰省，使鋝以告𬇊，迺俾[mask]以曶酒及羊、絲三鋝，用致茲人，曶迺誨于𬇎[mask]：汝其舍究矢五秉，曰：弋當俾處厥邑，田[mask]田，𬇊則俾復令曰：諾。昔饉歲，匡眾厥臣廿夫，寇曶禾十秭，以[mask]季告東宮，東宮迺曰：求乃人，乃弗得，汝匡罰大，[mask]迺𩒨首，于曶用五田，用眾一夫，曰嗌，用臣曰疐、[mask]朏、曰奠，曰：用茲四夫，𩒨首曰：余無由具寇足[mask]，不出，鞭余，曶又以匡季告東宮，曶曰：弋唯朕[mask] [mask]償，東宮迺曰：償曶禾十秭，遺十秭，為廿秭，[mask] 來歲弗償，則付𠦌秭，迺或即曶：用田二，又臣[mask] [mask]，凡用即曶田七田，人五夫，曶覓匡卅秭。
"""
print(f"测试句子：{text}")

text = re.sub(r"\[(MASK|mask)\]", tokenizer.mask_token, text)

inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

mask_token_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=False)

with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

top_k = 10
print("\n预测结果：")
for idx, (batch_ix, pos_ix) in enumerate(mask_token_indices):
    mask_logits = logits[batch_ix, pos_ix, :]
    probs = torch.nn.functional.softmax(mask_logits, dim=-1)
    values, predictions = torch.topk(probs, top_k, dim=-1)
    print(f"\n第{idx+1}个MASK位置：")
    for pred_id, prob in zip(predictions.tolist(), values.tolist()):
        pred_char = tokenizer.convert_ids_to_tokens(pred_id)
        if pred_char.startswith("##"):
            pred_char = pred_char[2:]
        print(f"  候选字：{pred_char}，概率：{prob:.4f}")


测试句子：
唯王元年，六月既朢乙亥，王在周穆王大[mask]。[mask]若曰：曶，命女更乃且考𤔲卜事，賜汝赤緼[mask]、□，用事，王在𫑕㡴，井叔賜曶赤金、鬱，曶受休命于王，曶用茲金作朕文考宄伯䵼牛鼎，曶其[mask][mask]用祀，子子孫孫其永寶。唯王四月既生霸，辰在丁酉，井叔在異，為[mask] [mask]，使厥小子究，以限訟于井叔，我既贖汝五[mask] [mask]。父用匹馬、束絲，限許曰：𬇊則俾我償馬。效[mask] [mask]俾復厥絲束，𫎛、效父廼許𬥱，曰：于王參門，[mask] [mask] 木梬，用徵誕贖茲五夫，用百鋝，非出五夫，[mask] [mask] 䛌，廼𬇎有䛌眔𫦠金，井叔曰：裁，王人廼贖[mask] [mask]，不逆付，曶毋俾弍，曶則拜𩒨首，受茲五[mask]，曰𬯂、曰恒、曰劦、曰𪝶、曰省，使鋝以告𬇊，迺俾[mask]以曶酒及羊、絲三鋝，用致茲人，曶迺誨于𬇎[mask]：汝其舍究矢五秉，曰：弋當俾處厥邑，田[mask]田，𬇊則俾復令曰：諾。昔饉歲，匡眾厥臣廿夫，寇曶禾十秭，以[mask]季告東宮，東宮迺曰：求乃人，乃弗得，汝匡罰大，[mask]迺𩒨首，于曶用五田，用眾一夫，曰嗌，用臣曰疐、[mask]朏、曰奠，曰：用茲四夫，𩒨首曰：余無由具寇足[mask]，不出，鞭余，曶又以匡季告東宮，曶曰：弋唯朕[mask] [mask]償，東宮迺曰：償曶禾十秭，遺十秭，為廿秭，[mask] 來歲弗償，則付𠦌秭，迺或即曶：用田二，又臣[mask] [mask]，凡用即曶田七田，人五夫，曶覓匡卅秭。


预测结果：

第1个MASK位置：
  候选字：室，概率：0.6551
  候选字：宮，概率：0.0612
  候选字：門，概率：0.0430
  候选字：使，概率：0.0201
  候选字：館，概率：0.0107
  候选字：夫，概率：0.0094
  候选字：殿，概率：0.0086
  候选字：臣，概率：0.0081
  候选字：宅，概率：0.0058
  候选字：廟，概率：0.0053

第2个MASK位置：
  候选字：王，概率：0.6241
  候选字：尹，概率：0.0350
  候选字：公，概率：0.0229
  候选字：汝，概率：0.0171
  候选字：唯，概率：0.0147
  候选字：廼，概率：0.0088
  候选字：臣，